In [1]:
import utils 
import os

## run the block for wikidata

In [2]:
# base directory path
base_dir = r"c:/Users/yuche/OneDrive - TUM/phd/KGC exp/pilot-exp/exp-round1"
data_dir = os.path.join(base_dir, "data")
wikidata_dir = os.path.join(data_dir, "wikidata_tekgen")
wikidata_new_dir = os.path.join(data_dir, "wikidata_new")
# dbpedia_dir = os.path.join(data_dir, "dbpedia")

ontology_file_path = os.path.join(wikidata_dir, "ontologies", "2_music_ontology.json")
test_file_path = os.path.join(wikidata_dir, "test/ont_2_music_test.jsonl")
similarity_file_path = os.path.join(wikidata_dir, "baselines", "test_train_sent_similarity/ont_2_music_test_train_similarity.json")
train_file_path = os.path.join(wikidata_dir, "train/ont_2_music_train.jsonl")
ground_truth_file_path = os.path.join(wikidata_dir, "ground_truth/ont_2_music_ground_truth.jsonl")

In [3]:
output_folder_path = os.path.join(wikidata_new_dir, "n_rels_0_distractor")
os.makedirs(output_folder_path, exist_ok=True)  # Will not throw an error if the folder exists

In [4]:
ontology = utils.load_json(ontology_file_path)
tests = utils.load_jsonl(test_file_path)
test_train_similarity = utils.load_json(similarity_file_path)
train = utils.load_jsonl(train_file_path)
ground_truth = utils.load_jsonl(ground_truth_file_path)
print(f"ontology: {ontology}")
print(f"tests: {len(tests)}")
print(f"test_train_similarity: {len(test_train_similarity.keys())}")
print(f"train: {len(train)}")
print(f"ground_truth: {len(ground_truth)}")

ontology: {'title': 'Music Ontology', 'id': 'ont_2_music', 'concepts': [{'qid': 'Q5', 'label': 'human'}, {'qid': 'Q638', 'label': 'music'}, {'qid': 'Q2188189', 'label': 'musical work'}, {'qid': 'Q4830453', 'label': 'record producer'}, {'qid': 'Q207628', 'label': 'composed musical work'}, {'qid': 'Q36834', 'label': 'composer'}, {'qid': 'Q34770', 'label': 'language'}, {'qid': 'Q188451', 'label': 'music genre'}, {'qid': 'Q7390', 'label': 'voice'}, {'qid': 'Q164204', 'label': 'musicology'}, {'qid': 'Q746359', 'label': 'music industry'}, {'qid': 'Q482994', 'label': 'album'}, {'qid': 'Q618779', 'label': 'award '}], 'relations': [{'pid': 'P86', 'label': 'composer', 'domain': 'Q2188189', 'range': 'Q5'}, {'pid': 'P361', 'label': 'part of', 'domain': 'Q2188189', 'range': 'Q482994'}, {'pid': 'P676', 'label': 'lyrics by', 'domain': 'Q2188189', 'range': 'Q5'}, {'pid': 'P577', 'label': 'publication date', 'domain': 'Q2188189', 'range': ''}, {'pid': 'P407', 'label': 'language of work or name', 'domai

# generate n ground truth relation, prompts and save to the output file

In [5]:
output_folder_path = os.path.join(wikidata_new_dir, "n_rels_0_distractor")
os.makedirs(output_folder_path, exist_ok=True)  # Will not throw an error if the folder exists

In [6]:
for item in tests:
    id = item["id"]
    item["gt_relations"] = utils.get_n_ontology_relations(ground_truth, id, ontology)
    print(item["gt_relations"])

['lyrics by', 'composer']
['composer', 'producer', 'lyrics by', 'performer']
['composer', 'performer']
['producer', 'lyrics by', 'composer']
['part of', 'lyrics by', 'publication date', 'performer', 'composer']
['lyrics by', 'producer', 'publication date', 'composer']
['producer', 'lyrics by', 'composer']
['lyrics by', 'composer']
['part of', 'composer', 'lyrics by', 'performer']
['composer', 'performer']
['lyrics by', 'composer']
['publication date', 'composer']
['composer']
['composer', 'lyrics by', 'performer']
['composer', 'lyrics by', 'publication date', 'performer']
['composer']
['composer', 'performer']
['part of', 'lyrics by', 'publication date', 'composer']
['composer']
['composer']
['lyrics by', 'composer']
['part of', 'lyrics by', 'composer']
['composer']
['composer', 'performer']
['producer', 'lyrics by', 'composer']
['composer']
['lyrics by', 'composer']
['instrumentation', 'composer']
['composer', 'producer', 'lyrics by', 'performer']
['composer']
['lyrics by', 'composer'

In [7]:
new_test_file_path = os.path.join(output_folder_path, "ont_2_music_test.jsonl")
utils.save_jsonl(tests, new_test_file_path)

Saved JSONL data to c:/Users/yuche/OneDrive - TUM/phd/KGC exp/pilot-exp/exp-round1\data\wikidata_new\n_rels_0_distractor\ont_2_music_test.jsonl


In [8]:
zero_shot_prompts = []
for item in tests:
    prompt_item = {}
    prompt_item["id"] = item["id"]
    prompt_item["prompt"]= utils.prepare_prompt_zero_shot(item["sent"], item["gt_relations"], ontology)
    print(prompt_item["id"])
    print(prompt_item["prompt"])
    print("-------------------")
    zero_shot_prompts.append(prompt_item)

ont_2_music_test_1
Extract relational triplets from the sentence based on the provided ontology relations.
Use only the listed relations and ensure subjects and objects align with their specified restrictions.
Only return the triples in the format relation(subject, object), separated by commas. Do not include explanations, extra text, or comments. 

CONTEXT:


Ontology Relations: lyrics_by(musical work, human), composer(musical work, human), 

Test Sentence: Composed by lead guitarist Brian May, ' 39 is the fifth track on their fourth studio album A Night at the Opera.
Test Output: 
-------------------
ont_2_music_test_2
Extract relational triplets from the sentence based on the provided ontology relations.
Use only the listed relations and ensure subjects and objects align with their specified restrictions.
Only return the triples in the format relation(subject, object), separated by commas. Do not include explanations, extra text, or comments. 

CONTEXT:


Ontology Relations: compose

In [9]:
output_file_path = os.path.join(output_folder_path, "ont_2_music_n_rels_0_distractor_0_shot_prompts.jsonl")
utils.save_jsonl(zero_shot_prompts, output_file_path)

Saved JSONL data to c:/Users/yuche/OneDrive - TUM/phd/KGC exp/pilot-exp/exp-round1\data\wikidata_new\n_rels_0_distractor\ont_2_music_n_rels_0_distractor_0_shot_prompts.jsonl


# generate one shot example prompt

In [5]:
# read new test file if the previous block was not ran
new_test_file_path = os.path.join(output_folder_path, "ont_2_music_test.jsonl")
tests = utils.load_jsonl(new_test_file_path)
print(tests[0])

{'id': 'ont_2_music_test_1', 'sent': "Composed by lead guitarist Brian May, ' 39 is the fifth track on their fourth studio album A Night at the Opera.", 'gt_relations': ['lyrics by', 'composer']}


In [6]:
one_shot_prompts = []
for item in tests:
    prompt_item = {}
    prompt_item["id"] = item["id"]
    # get the similar train sentences for the test sentence
    similar_sents = utils.get_similar_sentences(item["id"], test_train_similarity)
    # we retrieve by default the first similar sentence from the list of similar sentences
    simil_sent_id = similar_sents[0]

    # we get the train sentence from the train sentences list and from there we process each field sub_label, obj_label, rel_label
    train_sent = utils.get_train_sentence(simil_sent_id, train)
    prompt_item["prompt"]= utils.prepare_prompt_one_shot(item["sent"], item["gt_relations"], ontology, train_sent)
    print(prompt_item["id"])
    print(prompt_item["prompt"])
    print("-------------------")
    one_shot_prompts.append(prompt_item)

ont_2_music_test_1
Extract relational triplets from the sentence based on the provided ontology relations and examples.
Use only the listed relations and ensure subjects and objects align with their specified restrictions.
Only return the triples in the format relation(subject, object), separated by commas. Do not include explanations, extra text, or comments. 

CONTEXT:


Ontology Relations: lyrics_by(musical work, human), composer(musical work, human), 

Example Sentence: Queen II was recorded at Trident Studios and Langham 1 Studios, London, in August 1973 with co-producers Roy Thomas Baker and Robin Geoffrey Cable, and engineered by Mike Stone.
Example Output: producer(Queen II,Roy Thomas Baker)

Test Sentence: Composed by lead guitarist Brian May, ' 39 is the fifth track on their fourth studio album A Night at the Opera.
Test Output: 
-------------------
ont_2_music_test_2
Extract relational triplets from the sentence based on the provided ontology relations and examples.
Use only

In [7]:
output_file_path = os.path.join(output_folder_path, "ont_2_music_n_rels_0_distractor_1_shot_prompts.jsonl")
utils.save_jsonl(one_shot_prompts, output_file_path)

Saved JSONL data to c:/Users/yuche/OneDrive - TUM/phd/KGC exp/pilot-exp/exp-round1\data\wikidata_new\n_rels_0_distractor\ont_2_music_n_rels_0_distractor_1_shot_prompts.jsonl


# generate three-shot examples prompt and save

In [8]:
three_shot_prompts = []
for item in tests:
    prompt_item = {}
    prompt_item["id"] = item["id"]
    # get the similar train sentences for the test sentence
    similar_sents = utils.get_similar_sentences(item["id"], test_train_similarity)
    # we retrieve by default the first three similar sentence from the list of similar sentences
    simil_sent_id = similar_sents[0:3]
    prompt_item["prompt"]= utils.prepare_prompt_three_shot(item["sent"], item["gt_relations"], ontology, simil_sent_id, train)
    print(prompt_item["id"])
    print(prompt_item["prompt"])
    print("-------------------")
    three_shot_prompts.append(prompt_item)

ont_2_music_test_1
Extract relational triplets from the sentence based on the provided ontology relations and examples.
Use only the listed relations and ensure subjects and objects align with their specified restrictions.
Only return the triples in the format relation(subject, object), separated by commas. Do not include explanations, extra text, or comments. 

CONTEXT:


Ontology Relations: lyrics_by(musical work, human), composer(musical work, human), 

Example Sentence: Queen II was recorded at Trident Studios and Langham 1 Studios, London, in August 1973 with co-producers Roy Thomas Baker and Robin Geoffrey Cable, and engineered by Mike Stone.
Example Output: producer(Queen II,Roy Thomas Baker)


Example Sentence: Written by bassist John Deacon, the song featured on the group's eighth studio album The Game (1980).
Example Output: lyrics_by(Another One Bites the Dust,John Deacon)


Example Sentence: The album had a chequered recording history, with work beginning in mid-1968, but t

In [9]:
output_file_path = os.path.join(output_folder_path, "ont_19_film_n_rels_0_distractor_3_shot_prompts.jsonl")
utils.save_jsonl(three_shot_prompts, output_file_path)

Saved JSONL data to c:/Users/yuche/OneDrive - TUM/phd/KGC exp/pilot-exp/exp-round1\data\wikidata_new\n_rels_0_distractor\ont_19_film_n_rels_0_distractor_3_shot_prompts.jsonl


# generate baseline relations with one shot prompt
### the instruct is changed, the onto class is removed, only keep the baseline onto relations and example

In [10]:
output_folder_path = os.path.join(wikidata_new_dir, "baseline_rels")
os.makedirs(output_folder_path, exist_ok=True)  # Will not throw an error if the folder exists

In [11]:
def get_concept_label(ontology, concept):
    """
    Get the label for the ontology concept
    :param ontology: ontology with concepts ids and labels
    :param ont_dom: input concept ID
    :return: the label of the input concept
    """
    for onto in ontology['concepts']:
        if onto['qid'] == concept:
            return onto['label']
        
        
def get_ontology_relations(ontology):
    """
    Generate a verbalized list of relations in the given ontology to be included in the prompt
    :param ontology:  an ontology as a dictionary
    :return: A string for ontology relations.
            e.g. cast_member(film,human), director (film,human), screenwriter(film,human), producer(film,human), ...
    """

    ont_rels = ""
    onto_rel_strings = list()
    for onto in ontology['relations']:
        ont_rel = onto['label']
        ont_rel = ont_rel.replace(" ", "_")
        ont_dom = onto['domain']
        ont_range = onto['range']
        ont_domain = get_concept_label(ontology, ont_dom)
        ont_range = get_concept_label(ontology, ont_range)

        if ont_rel == None:
            continue
        if ont_domain == None:
            ont_domain = ""
        if ont_range == None:
            ont_range = ""

        onto_rel_strings.append(f"{ont_rel} ({ont_domain},{ont_range})\n")

        ont_rels += ont_rel + "(" + ont_domain + "," + ont_range + "), "

    return ont_rels[0:-2]


def get_similar_sentences(test_sentence_id, test_similar):
    for sim in test_similar:
        if sim == test_sentence_id:
            return test_similar[sim]


def get_train_sentence(simil_sent_id, train_sentences):
    for sent in train_sentences:
        if sent['id'] == simil_sent_id:
            return sent


def get_example_prompt(train_sent):
    example_prompt = "\n\nExample Sentence: " + train_sent['sent']
    train_sent['rel_label'] = train_sent['rel_label'].replace(" ", "_")
    example_prompt += "\nExample Output: " + train_sent['rel_label'] + "(" + train_sent['sub_label'] + "," + train_sent['obj_label'] + ")"
    return example_prompt


def get_test_prompt(test_sentence):
    test_prompt = "\n\nTest Sentence: " + test_sentence
    test_prompt += "\nTest Output: "
    return test_prompt


def prepare_prompt(ontology: dict, test_sentence: str, train_sent: str) -> str:

    prompt_fixed = '''Extract relational triplets from the sentence based on the provided ontology relations and examples.
Use only the listed relations and ensure subjects and objects align with their specified restrictions.
Only return the triples in the format relation(subject, object), separated by commas. Do not include explanations, extra text, or comments. \n
'''

    prompt = prompt_fixed
    prompt += 'CONTEXT:\n\n'
    prompt += '\nOntology Relations: '
    prompt += get_ontology_relations(ontology)
    prompt += get_example_prompt(train_sent)
    prompt += get_test_prompt(test_sentence)

    return prompt

In [12]:
baseline_one_shot_prompts = []
for test_sentence in tests:
            test_sentence_id = test_sentence['id']
            # test sentence for which the prompt to be generated
            test_sentence = test_sentence['sent']

            # get the similar train sentences for the test sentence
            similar_sents = get_similar_sentences(test_sentence_id, test_train_similarity)
            # we retrieve by default the first similar sentence from the list of similar sentences
            simil_sent_id = similar_sents[0]

            # we get the train sentence from the train sentences list and from there we process each field sub_label, obj_label, rel_label
            train_sent = get_train_sentence(simil_sent_id, train)

            # prompt generation logic
            prompt = prepare_prompt(ontology, test_sentence, train_sent)
            prompt_data = {'id': test_sentence_id, 'prompt': prompt}
            baseline_one_shot_prompts.append(prompt_data)
            print(prompt_data)

{'id': 'ont_2_music_test_1', 'prompt': "Extract relational triplets from the sentence based on the provided ontology relations and examples.\nUse only the listed relations and ensure subjects and objects align with their specified restrictions.\nOnly return the triples in the format relation(subject, object), separated by commas. Do not include explanations, extra text, or comments. \n\nCONTEXT:\n\n\nOntology Relations: composer(musical work,human), part_of(musical work,album), lyrics_by(musical work,human), publication_date(musical work,), language_of_work_or_name(musical work,language), voice_type(human,voice), instrumentation(musical work,), tracklist(album,musical work), genre(musical work,music genre), performer(musical work,), producer(album,), nominated_for(album,award ), record_label(album,)\n\nExample Sentence: Queen II was recorded at Trident Studios and Langham 1 Studios, London, in August 1973 with co-producers Roy Thomas Baker and Robin Geoffrey Cable, and engineered by Mi

In [13]:
output_file_path = os.path.join(output_folder_path, "ont_2_music_baseline_rels_1_shot_prompts.jsonl")
utils.save_jsonl(baseline_one_shot_prompts, output_file_path)

Saved JSONL data to c:/Users/yuche/OneDrive - TUM/phd/KGC exp/pilot-exp/exp-round1\data\wikidata_new\baseline_rels\ont_2_music_baseline_rels_1_shot_prompts.jsonl
